# 1. TransUNet for 3D Medical Image

## Feature Size for Concatenation : 1/2, 1/4, 1/8
## Feature Size for Embedding : 1/16

HyperParameter

- Batch Size

- Learning Rate

- Loss Weights Rate

In [1]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
from tensorflow.keras.layers import Conv3D, BatchNormalization, ReLU, UpSampling3D

from tensorflow.python.keras.layers import Input
from tensorflow.keras import layers
#!pip install tensorflow-addons
import tensorflow_addons as tfa
import tensorflow as tf
import math

from matplotlib import pyplot as plt
#!pip install pydicom
import pydicom as dcm

import numpy as np
import random
#!pip install opencv-python
import cv2
import os

In [32]:
#!pip install torch-multi-head-attention

In [4]:
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
# 1. Data
image_width, image_height, image_depth = 512//2, 512//2, 64

image_channel = 1

base_dir = "body-morphometry-kidney-and-tumor/"
save_dir = "C:/Users/user/Desktop/body-morphometry-kidney-and-tumor/train/"

# 2. Model
# 2.1. Transformer
featuremap_size =  (image_width// 16, image_height// 16, image_depth // 16)

patch_size = 1
num_patches = featuremap_size[0] * featuremap_size[1] * featuremap_size[2]

HiddenSizeD = 768
num_heads = 12
TransformerLayer = 12

MLP_size = 3072
Transformer_MLP_unit = [MLP_size, HiddenSizeD]

ClassHead_MLP_unit = [MLP_size]

# 2.2. MLP
dropout_rate = 0.1
num_classes = 3

lr = 0.001
weight_decay = 0.0001
batch_size = 1
epochs = 300

In [6]:
print(f"1. featuremap Shape : {featuremap_size[0], featuremap_size[1], featuremap_size[2], image_channel}")
print(f"2. patch_size : {patch_size}")
print(f"3. Number of Patches : {num_patches}")
print(f"4. Dimension : {HiddenSizeD}")
print(f"5. Number of Heads : {num_heads}")
print(f"6. Number of Transformer Encedoer : {TransformerLayer}")
print(f"7. MLP size : {MLP_size}")

1. featuremap Shape : (16, 16, 4, 1)
2. patch_size : 1
3. Number of Patches : 1024
4. Dimension : 768
5. Number of Heads : 12
6. Number of Transformer Encedoer : 12
7. MLP size : 3072


In [7]:
# Weight Standardization
class WSConv3D(tf.keras.layers.Conv3D):
    def __init__(self, *args, **kwargs):
        super(WSConv3D, self).__init__(*args, **kwargs)

    def standardize_weight(self, kernel, epsillon):
        kernel_mean = tf.math.reduce_mean(kernel, axis=[0, 1, 2, 3], keepdims=True, name='kernel_mean')
        kernel = kernel - kernel_mean
        kernel_std = tf.keras.backend.std(kernel, axis=[0, 1, 2, 3], keepdims=True)
        return kernel / (kernel_std + epsillon)

    def call(self, inputs, epsillon=1e-4):
        self.kernel.assign(self.standardize_weight(self.kernel, epsillon))
        return super().call(inputs)

In [8]:
# Patch creation as a layer
class Patches(tf.keras.layers.Layer):
    def __init__(self, num_patches = num_patches):
        super(Patches, self).__init__()
        self.num_patches = num_patches

    def call(self, volume):
        # volume (None, 16, 16, 4, 1024)
        #print(f"volume shape : {tf.shape(volume)}")

        batch_size = tf.shape(volume)[0]
        #print(f"type batch_size : {type(batch_size)}")
        patches_dim = volume.shape[-1]
        #print(f"type patches_dim : {type(patches_dim)}")

        patches = tf.reshape(volume, shape = [batch_size, self.num_patches, patches_dim])
        #print(f"patch shape 1D : {patches.shape}") # (None, None, 1024)
        return patches

In [9]:
# patch encoder
class PatchEncoder(tf.keras.layers.Layer):
    """
    : Patch (driven from Linear Projection of Flattened Patches) + Position Embedding
    Linear Projection
    Position Embedding
    """
    def __init__(self, 
                 num_patches = num_patches, 
                 HiddenSizeD = HiddenSizeD,
                 ):
        # num_patches = 16 * 16 * 4 = 1024
        # HiddenSizeD = 768
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.HiddenSizeD = HiddenSizeD
        self.projection = layers.Dense(units = HiddenSizeD)
        
        # Positional Embedding : Turns positive integers (indexes) into dense vectors of fixed size.
        self.pos_emb = self.add_weight("pos_emb", shape=(1, num_patches, HiddenSizeD))
        

    def call(self, patch):
        # preprojection patch : (1024, 1025)
        patch = self.projection(patch)
        # postprojection patch : (1024, 768)
        return patch + self.pos_emb

In [10]:
# MLP
"""
GeLu : 0.5x(1 + tanh[p2/π(x + 0.044715x^3)])
더 빠르게 수렴된단다.
"""
def mlp(x, units, dropout_rate):
    for unit in units:
        x = layers.Dense(unit, activation = tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [11]:
class Conv3DReLu(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, norm, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.padding = "same"
        self.strides = 1
        self.norm = norm

    def build(self, input_shape):
        self.conv = WSConv3D(
            filters = self.filters, kernel_size = self.kernel_size, strides = self.strides,
            padding = self.padding, use_bias = False, kernel_regularizer = tf.keras.regularizers.L2(1e-4), 
            kernel_initializer = "lecun_normal")

        self.bn = BatchNormalization(momentum=0.9, epsilon=1e-5)
        self.gn = tfa.layers.GroupNormalization(epsilon=1e-5, groups = 16)

    def call(self, inputs):
        x = self.conv(inputs)
        if self.norm == "bn":
            x = self.bn(x)
        elif self.norm == "gn":
            x = self.gn(x)
        else:
            raise Exception("bn or gn")
        x = tf.nn.relu(x)
        return x

In [12]:
class DecoderBlock(tf.keras.layers.Layer):
    def __init__(self, filters, norm, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.norm = norm

    def build(self, input_shape):
        self.conv1 = Conv3DReLu(filters = self.filters, kernel_size = 3, norm = self.norm)
        self.conv2 = Conv3DReLu(filters = self.filters, kernel_size = 3, norm = self.norm)
        self.upsampling = UpSampling3D(size = 2)

    def call(self, inputs, skip = None):
        x = self.upsampling(inputs)
        if skip is not None:
            x = tf.concat([x, skip], axis=-1)
        x = self.conv1(x)
        x = self.conv2(x)
        return x

In [13]:
def CNN3D(inputs, norm):
    f1 = WSConv3D(filters = 64, kernel_size=3, activation="relu", padding = "same")(inputs)
    f1 = layers.MaxPool3D(pool_size = (2,2,2))(f1)
    if norm == "bn":
        f1 = layers.BatchNormalization()(f1) # 256
    elif norm == "gn":
        f1 = tfa.layers.GroupNormalization(epsilon=1e-5, groups = 16)(f1) # 256

    f2 = WSConv3D(filters = 256, kernel_size=3, activation="relu", padding = "same")(f1)
    f2 = layers.MaxPool3D(pool_size = (2,2,2))(f2)
    if norm == "bn":
        f2 = layers.BatchNormalization()(f2) # 128
    elif norm == "gn":
        f2 = tfa.layers.GroupNormalization(epsilon=1e-5, groups = 16)(f2)

    f3 = WSConv3D(filters = 512, kernel_size=3, activation="relu", padding = "same")(f2)
    f3 = layers.MaxPool3D(pool_size = (2,2,2))(f3)
    if norm == "bn":
        f3 = layers.BatchNormalization()(f3) # 64
    elif norm == "gn":
        f3 = tfa.layers.GroupNormalization(epsilon=1e-5, groups = 16)(f3)

    f4 = WSConv3D(filters = 1024, kernel_size=3, activation="relu", padding = "same")(f3)
    f4 = layers.MaxPool3D(pool_size = (2,2,2))(f4)
    if norm == "bn":
        f4 = layers.BatchNormalization()(f4) # 32
    elif norm == "gn":
        f4 = tfa.layers.GroupNormalization(epsilon=1e-5, groups = 16)(f4)
        
    return [f4, f3, f2, f1]

In [14]:
# Build the ViT model
def create_model(norm,
                 image_width = image_width,
                 image_height= image_height,
                 image_depth = image_depth,
                 image_channel = image_channel,
                 num_patches = num_patches,
                 num_heads = num_heads,
                 HiddenSizeD = HiddenSizeD,
                 Transformer_MLP_unit = Transformer_MLP_unit,
                 ClassHead_MLP_unit = ClassHead_MLP_unit
                 ):
    # 1. Encoder : (256, 256, 64, 1) -> (1024, 768)
    # 1.1. CNN
    # Input : (256, 256, 64, 1)
    # output : (16, 16, 4, 512)
    input_tensor = Input(shape=(image_width, image_height, image_depth, image_channel))
    
    """
    여기서 CNN Model이 들어가야한다.
    """
    feature_maps = CNN3D(input_tensor, norm) # 32
    
    # (16, 16, 4, 1024), (32, 32, 8, 512), (64, 64, 16, 256), (128, 128, 32, 64)

    # 1.2. Patch
    # input : (16, 16, 4, 1024)
    # output : (1024, 768)
    patches = Patches()(feature_maps[0]) # (1024, 1024)
    del feature_maps[0]
    encoded_patches = PatchEncoder()(patches) # (1024, 768)
    
    
    # 1.3. Transformer block.
    # input : (1024, 768)
    # output : (1024, 768)
    for _ in range(TransformerLayer):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = HiddenSizeD, dropout = dropout_rate
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
       
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, Transformer_MLP_unit, dropout_rate = dropout_rate)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])
    
    
    # 2. Bridge : (1024, 768) -> (16, 16, 4, 768) -> (32, 32, 8, 512)
    # 2.1. reshape
    # Input : (1024, 768)
    # output : (16, 16, 4, 768)
    encodded_feature = tf.keras.layers.Reshape(target_shape = [image_width//16,
                                                               image_height//16,
                                                               image_depth//16,
                                                               HiddenSizeD]
                                               )(encoded_patches)
    
    # 2.2. Conv3D
    # Input : (16, 16, 4, 768)
    # output : (16, 16, 4, 512)
    x = WSConv3D(filters = 512, kernel_size = 3, strides = 1, padding = "same",
               use_bias=False, kernel_regularizer = tf.keras.regularizers.L2(1e-4), 
               kernel_initializer="lecun_normal")(encodded_feature)
    if norm == "bn":
        x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    elif norm == "gn":
        x = tfa.layers.GroupNormalization(epsilon=1e-5, groups = 16)(x)
    x = tf.nn.relu(x)
    
    # 3. Decoder CUP: (16, 16, 4, 512) -> (32, 32, 8, 256) -> (64, 64, 16, 128) -> (128, 128, 32, 64) -> (256, 256, 64, 16)
    blocks = [DecoderBlock(filters = block_channel, norm = norm) 
              for block_channel in [256, 128, 64, 16]]
    
    for i, block in enumerate(blocks):
        skip = feature_maps[i] if (i < 3) else None
        x = block(x, skip = skip)
    
    
    # 4. Segmentation Head
    # Input : (256, 256, 64, 16)
    # output : (256, 256, 64, 3)
    x = WSConv3D(filters = num_classes, kernel_size = 1, padding = "same",
                 name = "segmentation_head_conv"
                )(x)
    
    model = tf.keras.Model(inputs = input_tensor, outputs = x, name = "TransUNet")
    return model
    #"""

model = create_model(norm = "bn")
model.summary(line_length = 150)

# 2. Dataset
## 2.1. Kaggle : Body Morphometry: Kidney and Tumor
####  https://www.kaggle.com/c/body-morphometry-kidney-and-tumor/discussion
### -> To segment Kidney and Tumor on CT



## 2.2. Data
 - Number of Data : 100 with 64 slices 
 - Size : (512, 512, 64)
 - HU : -1024 ~ 3096

In [15]:
train_x_dir = f"{base_dir}/train/DICOM/"
train_y_dir = f"{base_dir}/train/Label/"
train_x_folder_list = os.listdir(train_x_dir)
#print(train_x_folder_list)

train_y_folder_list = os.listdir(train_y_dir)
#print(train_y_folder_list)

In [16]:
def plot_slices(data, vmin = None, vmax = None, cmap = plt.cm.bone, dcm_flag = 0):
    f, axarr = plt.subplots(8, 8, figsize=(50, 50))
    cnt = 0
    for i in range(8):
        for j in range(8):
            if dcm_flag == 1:
                axarr[i, j].imshow(data[:, :, cnt], cmap=cmap, vmin=vmin, vmax=vmax)
            else:
                axarr[i, j].imshow(data[:, :, cnt, :], cmap=cmap, vmin=vmin, vmax=vmax)
            axarr[i, j].set_title(f'slice {cnt+1}', fontsize=20)
            axarr[i, j].axis("off")
            cnt +=1
    
    plt.show()

for i, folder in enumerate(train_x_folder_list):
    # 열어서 파일 list 만들고
    files_list = os.listdir(f"{train_x_dir}{folder}/")
    images = np.zeros([512, 512, 64])
    
    for j, file in enumerate(files_list):
        # 각 파일은 pydicom으로 열기
        image_dir = f"{train_x_dir}{folder}/{file}"
        image = dcm.read_file(image_dir).pixel_array
        images[:, :, j] = image

    plot_slices(images, dcm_flag = 1)

    if i== 0:
        break

for i, folder in enumerate(train_y_folder_list):
    # 열어서 파일 list 만들고
    files_list = os.listdir(f"{train_y_dir}{folder}/")
    labels = np.zeros([512, 512, 64])
    
    for j, file in enumerate(files_list):
        # 각 파일은 pydicom으로 열기
        label_dir = f"{train_y_dir}{folder}/{file}"
        label = cv2.imread(label_dir, 0)
        labels[:, :, j] = label
    
    plot_slices(labels, 0, 2, cmap = None, dcm_flag = 1)
    if i== 0:
        break

## 2.3. Preprocessing
 - Image Scaling : HU to Grayscale
 - Data Augmentation : 최대한 많이

### 2.3.1. Image Scaling and Save
#### Window Level : 30
#### Window Width : 400 

In [25]:
import PIL
PIL.__version__

'8.2.0'

In [23]:
tf.__version__

'2.3.0'

In [24]:
numpy.__version__

'1.18.5'

In [26]:
wl, ww = 30, 400


for i, folder in enumerate(train_y_folder_list):
    # 열어서 파일 list 만들고

    files_list = os.listdir(f"{train_y_dir}{folder}/")

    #masks = np.zeros([image_height, image_width, image_depth, num_classes], dtype = np.float32)
 
    masks = np.zeros([image_height, image_width, image_depth, image_channel], dtype = np.float32)
   
    volume = np.zeros([image_height, image_width, image_depth, image_channel], dtype = np.uint8)
    
    for j, file in enumerate(files_list):
        file_name, file_format = file.split(".")
        label_dir = f"{train_y_dir}{folder}/{file_name}.png"
        label = cv2.imread(label_dir, 0)
        label = cv2.resize(label, dsize=(image_width, image_height), interpolation=cv2.INTER_AREA)

        # 1. label
        label[label < 0.5] = 0
        label[label >= 1.5] = 2
        label[(label >= 0.5) & (label < 1.5)] = 1
        """
        label = tf.one_hot(label,
                           depth=num_classes,
                           dtype=tf.uint8,
                          )
    
        label = tf.reshape(label, [image_height, image_width, num_classes])
        """

        label = tf.reshape(label, [image_height, image_width, image_channel])
        masks[:,:,j,:] = label
        print(3)

        # 2. image
        image_dir = f"body-morphometry-kidney-and-tumor/train/DICOM/{folder}/{file_name}.dcm"
        raw_image = dcm.read_file(image_dir).pixel_array
        raw_image = cv2.resize(raw_image, dsize=(image_width, image_height), interpolation=cv2.INTER_AREA)
        image = np.clip(raw_image, wl - (ww / 2), wl + (ww / 2))
        max_v, min_v = image.max(), image.min()
        image = (image - min_v) / (max_v - min_v) * 255
        image = np.reshape(image , [image_height, image_width, image_channel])
        volume[:,:,j,:] = image

    np.save(f"C:/Users/user/Desktop/body-morphometry-kidney-and-tumor/train/Data_3/patient_{i+1}", volume)
    np.save(f"C:/Users/user/Desktop/body-morphometry-kidney-and-tumor/train/Mask_3/patient_{i+1}", masks)
    print(f"Patient{i+1} complete")

print("Complete")

TypeError: __array__() takes 1 positional argument but 2 were given

In [34]:
pip list

Package                            Version
---------------------------------- -------------------
-illow                             7.2.0
-umpy                              1.18.5
absl-py                            0.15.0
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 1.9.12Note: you may need to restart the kernel to use updated packages.
anaconda-project                   0.8.3
argh                               0.26.2
asn1crypto                         1.3.0
astroid                            2.4.2
astropy                            4.0.1.post1
astunparse                         1.6.3
atomicwrites                       1.4.0
attrs                              19.3.0
autopep8                           1.5.3
Babel                              2.8.0
backcall                           0.2.0
backports.functools-lru-cache      1.6.1
backports.shutil-get-terminal-size 1.0.0
backports.tempfile                 1.0
backpor

### 2.3.2. Data Augmentation

하는 방법 찾기

- Flip (Vertical, Horizontal)

- Random Rotation -170 ~ +170 (10)

- Translate

- Gaussian Noise

- Random Shear

- Gamma Contrast

- Gaussian_blur

--> x9

In [19]:
!pip install imgaug
import imgaug.augmenters as iaa
import skimage as sk
import cv2

# 1. Flip
def Flip(data, mask, flag,
         width = image_width,
         height = image_height,
         depth = image_depth,
         channel = image_channel,
         num_classes = num_classes
        ):
    aug_data = np.zeros([width, height, depth, channel], dtype = np.float32)
    aug_mask = np.zeros([width, height, depth, channel], dtype = np.float32)
    if flag == "lr":
        for i in range(depth):
            aug_data[:, :, i, :] = np.reshape(cv2.flip(data[:, :, i, :], 1), [width, height, channel])
            aug_mask[:, :, i, :] = np.reshape(cv2.flip(mask[:, :, i, :], 1), [width, height, channel])
    else:
        for i in range(depth):
            aug_data[:, :, i, :] = np.reshape(cv2.flip(data[:, :, i, :], 0), [width, height, channel])
            aug_mask[:, :, i, :] = np.reshape(cv2.flip(mask[:, :, i, :], 0), [width, height, channel])
    
    aug_mask = np.eye(num_classes)[aug_mask.astype(int)].astype('float32')
    aug_mask = np.reshape(aug_mask, [height, width, depth, num_classes])
    return aug_data, aug_mask

    

# 2. Rotation (-170 ~ +170)
def Rotation(data, mask,
             width = image_width,
             height = image_height,
             depth = image_depth,
             channel = image_channel,
             num_classes = num_classes
            ):
    
    aug_data = np.zeros([width, height, depth, channel], dtype = np.float32)
    aug_mask = np.zeros([width, height, depth, channel], dtype = np.float32)
    
    rot_matrix = cv2.getRotationMatrix2D((int(width/2), int(height/2)), int(random.uniform(-17, 17)) *10, 1)
    
    for i in range(depth):
        aug_data[: ,:, i, :] = np.reshape(cv2.warpAffine(data[: ,:, i, :], rot_matrix, (width, height)), [width, height, channel])
        aug_mask[: ,:, i, :] = np.reshape(cv2.warpAffine(mask[: ,:, i, :], rot_matrix, (width, height)), [width, height, channel])
    
    aug_mask[aug_mask < 0.5] = 0
    aug_mask[aug_mask >= 1.5] = 2
    aug_mask[(aug_mask >= 0.5) & (aug_mask < 1.5)] = 1
    
    aug_mask = np.eye(num_classes)[aug_mask.astype(int)].astype('float32')
    aug_mask = np.reshape(aug_mask, [height, width, depth, num_classes])
    return aug_data, aug_mask


# 3. Translate
def Translate(data, mask,
              width = image_width,
              height = image_height,
              depth = image_depth,
              channel = image_channel,
              num_classes = num_classes
             ):
    aug_data = np.zeros([width, height, depth, channel], dtype = np.float32)
    aug_mask = np.zeros([width, height, depth, channel], dtype = np.float32)
    
    x_shift = random.uniform(-width//2, width//2)
    y_shift = random.uniform(-height//2, height//2)
    
    shift_matrix = np.float32([[1, 0, x_shift],
                               [0, 1, y_shift]])  
    
    for i in range(depth):
        aug_data[:, :, i, :] = np.reshape(cv2.warpAffine(data[:, :, i, :], shift_matrix, (width, height)), [width, height, channel])
        aug_mask[:, :, i, :] = np.reshape(cv2.warpAffine(mask[:, :, i, :], shift_matrix, (width, height)), [width, height, channel])
    
    aug_mask[aug_mask < 0.5] = 0
    aug_mask[aug_mask >= 1.5] = 2
    aug_mask[(aug_mask >= 0.5) & (aug_mask < 1.5)] = 1
    
    aug_mask = np.eye(num_classes)[aug_mask.astype(int)].astype('float32')
    aug_mask = np.reshape(aug_mask, [height, width, depth, num_classes])
    return aug_data, aug_mask


# 4. Random Shear
def Shear(data, mask,
          width = image_width,
          height = image_height,
          depth = image_depth,
          channel = image_channel,
          num_classes = num_classes
         ):
    rand_deg = np.random.randint(-10, 10)
    shear = iaa.Affine(shear=rand_deg)
    aug_data = shear.augment_images(data)
    aug_mask = shear.augment_images(mask)
    
    aug_mask[aug_mask < 0.5] = 0
    aug_mask[aug_mask >= 1.5] = 2
    aug_mask[(aug_mask >= 0.5) & (aug_mask < 1.5)] = 1
    
    aug_mask = np.eye(num_classes)[aug_mask.astype(int)].astype('float32')
    aug_mask = np.reshape(aug_mask, [height, width, depth, num_classes])
    return aug_data, aug_mask

#################################################################################
# 5. Gaussian Noise
def GaussianNoise(data, mask,
                  width = image_width,
                  height = image_height,
                  depth = image_depth,
                  channel = image_channel,
                  num_classes = num_classes
                 ):
    #data = data/ 255.
    aug_data = sk.util.random_noise(data, mode='gaussian')
    
    #aug_mask = np.eye(num_classes)[mask.astype(int)].astype('float32')
    #aug_mask = np.reshape(aug_mask, [height, width, depth, num_classes])
    return aug_data, mask


# 6. Gamma Contrast
def GammaContrast(data, mask,
                  min_range=0.5, max_range=2.0,
                  width = image_width,
                  height = image_height,
                  depth = image_depth,
                  channel = image_channel,
                  num_classes = num_classes
                 ):
    rand_gamma = np.random.uniform(min_range, max_range)
    gamma = iaa.Sequential(iaa.GammaContrast(rand_gamma))
    aug_data = gamma.augment_images(data)
    
    #aug_mask = np.eye(num_classes)[mask.astype(int)].astype('float32')
    #aug_mask = np.reshape(aug_mask, [height, width, depth, num_classes])
    return aug_data, mask

# 7. Gaussian_blur
def GaussianBlur(data, mask,
                 width = image_width,
                 height = image_height,
                 depth = image_depth,
                 channel = image_channel,
                 num_classes = num_classes
                ):
    rand_sigma = np.random.uniform(0.0, 3.0)
    blur = iaa.GaussianBlur(sigma=rand_sigma)
    aug_data = blur.augment_images(data)
    
    #aug_mask = np.eye(num_classes)[mask.astype(int)].astype('float32')
    #aug_mask = np.reshape(aug_mask, [height, width, depth, num_classes])
    return aug_data, mask

  Using cached imgaug-0.4.0-py2.py3-none-any.whl (948 kB)
  Using cached Shapely-1.8.1.post1-cp38-cp38-win_amd64.whl (1.3 MB)


In [24]:
for i in range(1, 101):
    src = np.load(f"{save_dir}Data_3/patient_{i}.npy")
    src = src / 255.
    np.save(f"{save_dir}Data_3/patient_{i}", src)
    
    target = np.load(f"{save_dir}Mask_3/patient_{i}.npy")
    
    # 1. Left-Right Flip
    tmp_src, tmp_target = Flip(src, target, 'lr')
    np.save(f"{save_dir}Data_3/patient_{i}_lrflip", tmp_src)
    np.save(f"{save_dir}Mask_3/patient_{i}_lrflip", tmp_target)
    #print(f"{i} - LR flip : {tmp_target.shape}")
    
    # 2. Up-Down Flip
    tmp_src, tmp_target = Flip(src, target, 'ud')
    np.save(f"{save_dir}Data_3/patient_{i}_udflip", tmp_src)
    np.save(f"{save_dir}Mask_3/patient_{i}_udflip", tmp_target)
    #print(f"{i} - UD flip : {tmp_target.shape}")
    
    # 3. Rotation
    tmp_src, tmp_target = Rotation(src, target)
    np.save(f"{save_dir}Data_3/patient_{i}_rot", tmp_src)
    np.save(f"{save_dir}Mask_3/patient_{i}_rot", tmp_target)
    #print(f"{i} - Rotation flip : {tmp_target.shape}")
    
    # 4. Translate
    tmp_src, tmp_target = Translate(src, target)
    np.save(f"{save_dir}Data_3/patient_{i}_translate", tmp_src)
    np.save(f"{save_dir}Mask_3/patient_{i}_translate", tmp_target)
    #print(f"{i} - Translate flip : {tmp_target.shape}")
    
    # 5.Random Shear
    tmp_src, tmp_target = Shear(src, target)
    np.save(f"{save_dir}Data_3/patient_{i}_shear", tmp_src)
    np.save(f"{save_dir}Mask_3/patient_{i}_shear", tmp_target)
    #print(f"{i} - Shear flip : {tmp_target.shape}")
    
    #############################################################
    target = np.eye(num_classes)[target.astype(int)].astype('float32')
    target = np.reshape(target, [image_height, image_width, image_depth, num_classes])
    np.save(f"{save_dir}Mask_3/patient_{i}", target)
    
    # 6. Gaussian Noise
    tmp_src, _ = GaussianNoise(src, target)
    np.save(f"{save_dir}Data_3/patient_{i}_noise", tmp_src)
    np.save(f"{save_dir}Mask_3/patient_{i}_noise", target) 
    
    # 7. Gamma Contrast
    tmp_src, _ = GammaContrast(src, target, min_range = 0.99, max_range = 1.01) # 3
    np.save(f"{save_dir}Data_3/patient_{i}_gamma", tmp_src)
    np.save(f"{save_dir}Mask_3/patient_{i}_gamma", target)
    
    # 8. Gaussian_blur
    tmp_src, _ = GaussianBlur(src, target)
    np.save(f"{save_dir}Data_3/patient_{i}_blur", tmp_src)
    np.save(f"{save_dir}Mask_3/patient_{i}_blur", target)
    print(f"Patient{i} complete")

print("Complete")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/user/Desktop/body-morphometry-kidney-and-tumor/train/Data_3/patient_1.npy'

data = np.load(f"{save_dir}Data_3/patient_1.npy")
plot_slices(data)

mask = np.load(f"{save_dir}Mask_3/patient_1.npy")
print(mask.shape)
#plot_slices(mask)

data = np.load(f"{save_dir}Data_3/patient_1_udflip.npy")
plot_slices(data)

data = np.load(f"{save_dir}Data_3/patient_1_lrflip.npy")
plot_slices(data)

data = np.load(f"{save_dir}Data_3/patient_1_rot.npy")
plot_slices(data)

data = np.load(f"{save_dir}Data_3/patient_1_translate.npy")
plot_slices(data)

mask = np.load(f"{save_dir}Mask_3/patient_1_translate.npy")
print(mask.shape)
#plot_slices(mask)

data = np.load(f"{save_dir}Data_3/patient_1_shear.npy")
plot_slices(data)

data = np.load(f"{save_dir}Data_3/patient_1_noise.npy")
plot_slices(data)

data = np.load(f"{save_dir}Data_3/patient_1_gamma.npy")
plot_slices(data)

data = np.load(f"{save_dir}Data_3/patient_1_blur.npy")
plot_slices(data)

### Dataset Train : Test : Valid = 8 : 1 : 1

### By Generator

In [69]:
name_list = os.listdir(f"{base_dir}/train/Data_3/")
#print(name_list)
print(name_list.index("patient_8.npy"), name_list.index("patient_9.npy"))
train_name = name_list[:name_list.index("patient_8.npy")]
valid_name = name_list[name_list.index("patient_8.npy") : name_list.index("patient_9.npy")]
test_name = name_list[name_list.index("patient_9.npy") :]
print(f"Number of Trainset : {len(train_name)}")
print(f"Number of Validnset : {len(valid_name)}")
print(f"Number of Testset : {len(test_name)}")
print(f"Total : {len(train_name) + len(valid_name) + len(test_name)}")

702 801
Number of Trainset : 702
Number of Validnset : 99
Number of Testset : 99
Total : 900


In [70]:
def train_generator(names = train_name):
    for name in names:
        x_dir = f"{base_dir}/train/Data_3/{name}"
        y_dir = f"{base_dir}/train/Mask_3/{name}"

        x = np.load(x_dir)
        y = np.load(y_dir)
        yield (x, y)

train_dataset = tf.data.Dataset.from_generator(train_generator,
                                               (tf.float32, tf.float32),
                                               (tf.TensorShape([image_height, image_width, image_depth, image_channel]), tf.TensorShape([image_height, image_width, image_depth, num_classes])),
                                              )
train_dataset = train_dataset.batch(batch_size)

In [71]:
def valid_generator(names = valid_name):
    for name in names:
        x_dir = f"{base_dir}/train/Data_3/{name}"
        y_dir = f"{base_dir}/train/Mask_3/{name}"

        x = np.load(x_dir)
        y = np.load(y_dir)
        yield (x, y)
        
valid_dataset = tf.data.Dataset.from_generator(valid_generator,
                                               (tf.float32, tf.float32),
                                               (tf.TensorShape([image_height, image_width, image_depth, image_channel]), tf.TensorShape([image_height, image_width, image_depth, num_classes])),
                                              )
valid_dataset = valid_dataset.batch(batch_size)

# 3. Training
### Model : TransUNet

In [72]:
def Dice_per_class(y_true, y_pred):
    smooth = 0.0001
    #y_pred = tf.cast(y_pred > 0.5, tf.float32)
    return (2 * tf.reduce_sum(y_true * y_pred) + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

In [73]:
def Dice(y_true, y_pred, flag = 0, num_classes = num_classes):
    y_pred = tf.nn.softmax(y_pred)
    dice = 0.0
    
    for class_idx in range(num_classes):
        if flag == 0:
            # (None, 256, 256, 64, 3)
            dice += Dice_per_class(y_true[:, :, :, :, class_idx], y_pred[:, :, :, :, class_idx])
        else:
            dice += Dice_per_class(y_true[:, :, :, class_idx], y_pred[:, :, :, class_idx])
    return dice / num_classes

In [74]:
class DiceLoss(tf.losses.Loss):
    def __init__(self):
        super(DiceLoss, self).__init__(reduction = 'auto', name = "DiceLoss")
        
    def call(self, y_true, y_pred):
        dice = Dice(y_true, y_pred)
        return 1-dice

class FocalTverskyLoss(tf.losses.Loss):
    def __init__(self, alpha = 0.7, gamma = 1, smooth = 1e-6):
        super(FocalTverskyLoss, self).__init__(reduction = 'auto', name = "FocalTverskyLoss")
        self.alpha = alpha
        self.gamma = gamma
        self.smooth = smooth
        
    def call(self, y_true, y_pred):
        y_pred = K.flatten(y_pred)
        y_true = K.flatten(y_true)

        #True Positives, False Positives & False Negatives
        TP = K.sum((y_pred * y_true))
        FP = K.sum(((1-y_true) * y_pred))
        FN = K.sum((y_true * (1-y_pred)))

        Tversky = (TP + self.smooth) / (TP + self.alpha * FP + (1-self.alpha) * FN + self.smooth)  
        
        return K.pow((1 - Tversky), self.gamma)

class CategoricalFocalLoss(tf.losses.Loss):
    """
    식 : loss = - y_true * alpha * ((1 - y_pred)^gamma) * log(y_pred)
        
    alpha: the same as weighting factor in balanced cross entropy, default 0.25
    gamma: focusing parameter for modulating factor (1-p), default 2.0
    """
    def __init__(self, gamma=2.0, alpha=0.25, num_classes = num_classes):
        super(CategoricalFocalLoss, self).__init__(reduction = 'auto', name = "CategoricalFocalLoss")
        self._gamma = gamma
        self._alpha = np.array([[alpha for _ in range(num_classes) ]], dtype = np.float32)
        self._epsilon = 1e-07
        
    def call(self, y_true, y_pred):
        y_pred = tf.nn.softmax(y_pred)
        y_pred = K.clip(y_pred, self._epsilon, 1. - self._epsilon)
        cross_entropy = -y_true * K.log(y_pred)
        loss = self._alpha * K.pow(1 - y_pred, self._gamma) * cross_entropy
        return K.mean(K.sum(loss, axis = -1))

class CategoricalCrossEntropy(tf.losses.Loss):
    def __init__(self):
        super(CategoricalCrossEntropy, self).__init__(reduction = 'auto', name = "CategoricalCrossEntropy")
        
    def call(self, y_true, y_pred):
        y_pred = tf.nn.softmax(y_pred)
        return tf.keras.losses.CategoricalCrossentropy(from_logits=True)(y_true, y_pred)

In [75]:
def run_training(norm, batch_size = batch_size, epochs = epochs):
    model = create_model(norm = norm)
    loss_fn = DiceLoss()
    #loss_fn = CategoricalFocalLoss()
    #loss_fn = CategoricalCrossEntropy()

    optimizer = tfa.optimizers.AdamW(
        learning_rate=lr, weight_decay=weight_decay
    )
    model.compile(
        optimizer = optimizer,
        loss = loss_fn,
        metrics=["accuracy", Dice],
    )

    
    callbacks_list = [tf.keras.callbacks.ModelCheckpoint(filepath = f"C:/Users/user/Desktop/TransUNet/TransUNet_3D/TransUNet_3D_{norm}" + "{epoch}_.h5",
                                                         monitor = "val_Dice",
                                                         save_best_only = True,
                                                         save_weights_only = True,
                                                         verbose = 1,
                                                         mode = "max"
                                                        )
                     ]

    history = model.fit(
        train_dataset,
        validation_data = valid_dataset,
        batch_size = batch_size,
        epochs = epochs,
        callbacks = callbacks_list,
        shuffle=True,
        verbose = 1
    )

    return history

In [85]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


In [86]:
history = run_training(norm = "bn")

AttributeError: module 'tensorflow.keras.layers' has no attribute 'MultiHeadAttention'

# 4. Test

In [38]:
new_test_name = []
for name in test_name:
    if ("blur" not in name) and ("gamma" not in name) and ("flip" not in name) and ("noise" not in name) and ("rot" not in name) and ("shear" not in name) and ("translate" not in name):
        new_test_name.append(name)
        
print(len(new_test_name),"\n",new_test_name)
test_name = new_test_name
del new_test_name

NameError: name 'test_name' is not defined

In [ ]:
def test_generator(names = test_name):
    for name in names:
        x_dir = f"{base_dir}/train/Data_3/{name}"
        y_dir = f"{base_dir}/train/Mask_3/{name}"

        x = np.load(x_dir)
        y = np.load(y_dir)
        yield (x, y)

test_dataset = tf.data.Dataset.from_generator(test_generator,
                                               (tf.float32, tf.float32),
                                               (tf.TensorShape([image_height, image_width, image_depth, image_channel]), tf.TensorShape([image_height, image_width, image_depth, num_classes])),
                                             )
test_dataset = test_dataset.batch(batch_size)

model = create_model()
loss_fn = DiceLoss()

optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=weight_decay)

model.compile(optimizer = optimizer,
              loss = loss_fn,
              metrics=["accuracy", Dice])

model.load_weights(f"C:/Users/CBNUH/Desktop/Transformer/2. TransUNet/TransUNet_3D/TransUNet_3D.h5")
_, _, dice = model.evaluate(test_dataset, verbose = 1)
print(f"Test dice: {round(dice * 100, 2)}%\n")

In [ ]:
test_dataset = tf.data.Dataset.from_generator(test_generator,
                                              (tf.float32, tf.float32),
                                              (tf.TensorShape([image_height, image_width, image_depth, image_channel]), tf.TensorShape([image_height, image_width, image_depth, num_classes])),
                                             )

model.load_weights(f"C:/Users/CBNUH/Desktop/Transformer/2. TransUNet/TransUNet_3D/TransUNet_3D.h5")

for i, test in enumerate(test_dataset):
    data, mask = test[0], test[1]
    plot_slices(data)
    data = tf.reshape(data, [1, image_width, image_height, image_depth, image_channel])
    
    mask = tf.math.argmax(mask, -1)
    mask = tf.reshape(mask, [image_width, image_height, image_depth, image_channel])
    plot_slices(mask, vmin = 0, vmax = 2)
    
    pred = model.predict(data)
    pred = pred[0]
    pred = tf.math.argmax(pred, -1)
    pred = tf.reshape(pred, [image_width, image_height, image_depth, image_channel])
    plot_slices(pred, vmin = 0, vmax = 2)

    if i == 6:
        break